## A model build using TweetBERT to classify tweet as causal or non-causal 

In [1]:
import pandas as pd
import numpy as np
import spacy 
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import random
import os
import torch.nn.functional as F
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import transformers
from tqdm import tqdm, trange
#from google.colab import drive, files
import io

In [2]:
#uploaded = files.upload()
#data = pd.read_excel(io.BytesIO(uploaded['Causality + hypoglycemia.xlsx']), sheet_name=">5000_samples_")


# data = pd.read_excel("/home/adrian/Downloads/Causality + hypoglycemia.xlsx", sheet_name=">5000_samples_")
data = pd.read_excel("Causality + hypoglycemia.xlsx", sheet_name=">5000_samples_")

<ipython-input-2-19ae4d9dded0>:6: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  data = pd.read_excel("Causality + hypoglycemia.xlsx", sheet_name=">5000_samples_")


In [3]:
print("Total count:", data.shape[0])
data = data[data["Causal association"].notnull()]
print("Labeled count:", data.shape[0])

data.head()

Total count: 5456
Labeled count: 4999


,id,text,full_text,Intent,Cause,Effect,Causal association,Charline association0=no;1=yes,Remarks
0,908171203029868545,"tonight , I learned my older girl will back he...","tonight , I learned my older girl will back he...",NaN,NaN,NaN,0.0,NaN,NaN
1,1203645589214367745,USER USER I knew diabetes and fibromyalgia wer...,USER USER I knew diabetes and fibromyalgia wer...,joke,NaN,NaN,0.0,NaN,NaN
2,1310596731063525376,⬇ ️ ⬇ ️ ⬇ ️ THIS ⬇ ️ ⬇ ️ ⬇ ️ My wife has type ...,⬇ ️ ⬇ ️ ⬇ ️ THIS ⬇ ️ ⬇ ️ ⬇ ️ My wife has type ...,mS,NaN,NaN,0.0,NaN,NaN
3,1125198453167022085,USER Cheers ! Have one for this diabetic too !,USER Cheers ! Have one for this diabetic too !,mS,NaN,NaN,0.0,NaN,NaN
4,1248600944138268673,USER Additionally the medicines are being char...,USER Additionally the medicines are being char...,NaN,medicines are being charged at MRP,costing much higher,1.0,NaN,NaN


### Interrater-reliabilty measure

In [4]:
from sklearn.metrics import cohen_kappa_score

charline = data[data["Charline association0=no;1=yes"].notnull()]
coder1 = charline["Causal association"].values
coder2 = charline["Charline association0=no;1=yes"]
score = cohen_kappa_score(coder1,coder2)
print('Cohen\'s Kappa:',score)

Cohen's Kappa: 0.8184494602551521


### Preprocessing

In [5]:
data["Causal association"].value_counts()

0.0    3719
1.0    1280
Name: Causal association, dtype: int64

In [6]:
# Normalisation for BertTweet
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re

tokenizer = TweetTokenizer()

# https://huggingface.co/vinai/bertweet-base
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):

    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

In [7]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    sentences = text.split("<stop>")
    #sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    sentences = [s  for s in sentences if s != ""]
    return sentences


def create_training_data(data):
    tweets = []
    causal_labels = []
    
    for i, row in data.iterrows():
#        print("\n")
#        print(row["full_text"])
#        print(row["Intent"], "->", row["Causal association"])
       
        sentences = split_into_sentences(row["full_text"])
        intents = set(str(row["Intent"]).strip().split(";"))
        causes = str(row["Cause"]).strip().split(";")
        effects = str(row["Effect"]).strip().split(";")
#        print("\tintents:", intents)
#        print("\tcauses: '{}'".format(causes))
#        print("\teffects: '{}'".format(effects))
        
        if set({"nan"}) == intents or set({" "}) == intents: # single sentence
#            print("\tA: add => causality: {}".format(row["Causal association"]))
            tweets.append(row["full_text"])
            causal_labels.append(row["Causal association"])

        elif (
             set({"q"}) == intents 
          or set({"joke"}) == intents 
          or set({"q", "joke"}) == intents
          or set({"joke", "mS"}) == intents 
          or set({"neg"}) == intents 
          or set({"neg", "msS"}) == intents
          or set({"neg", "mS"}) == intents
          or set({"neg", "msS", "mE"}) == intents
          or set({"q", "joke", "mS"}) == intents
          or set({"q", "msS", "neg"}) == intents
          or set({"neg", "mC"}) == intents
          or set({"mC", "joke", "msS"}) == intents
          or set({"joke", "mE"}) == intents
        ):
#            print("\tB ignore")
            continue
        elif (  
             set({"mS"}) == intents # multiple sentences (possible that cause and effect in different sentences -> ignore)
          or set({"q", "mS"}) == intents # multiple sentences or question
          or set({"mS", "mE"}) == intents
          or set({"mC", "mS"}) == intents
          or set({"mC", "mS", "mE"}) == intents
          or set({"q", "mC", "mS"}) == intents
          or set({"q", "mC", "mS", "mE"}) == intents
            
        ):
            for sent in sentences:
#                print(sent)
                if sent[-1] != "?": # ignore questions
#                    print("\tC: add, causality => 0")
                    tweets.append(sent)
                    causal_labels.append(0)  
#                else:
#                    print("\tD: ignore")
        elif (
            set({"msS"}) == intents # multiple sentences with cause and effect in single sentence
         or set({"q", "msS"}) == intents # msS and a question
         or set({"msS", "mE"}) == intents # msS with several effects
         or set({"mC", "msS"}) == intents
         or set({"mE"}) == intents # multiple effects
         or set({"mC"}) == intents # multiple causes
         or set({"mC", "msS", "mE"}) == intents
         or set({"mC", "mE"}) == intents
         or set({"q", "mC", "mE"}) == intents   
         or set({"q", "mC", "msS"}) == intents
        ):

            if row["Causal association"] != 1: #TEST
                print(sentences)
                print("1) ERROR: Causal association should be 1 !!!!")      
                print()
        
            for sent in sentences:
#                print("sub sent:", sent)
                if sent[-1] != "?": # ignore question
                    
                    existCause = False
                    for cause in causes:
                        if cause in sent:
#                            print("Cause: <{}> in sent".format(cause))
                            existCause = True
                    
                    existEffect = False
                    for effect in effects:
                        if effect in sent:
#                            print("Effect: <{}> in sent".format(effect))
                            existEffect = True
                            
                    if existCause and existEffect:
                        tweets.append(sent)
                        causal_labels.append(row["Causal association"])
#                        print("E: add with Cause + effect => association: {}".format(row["Causal association"]))
                    else:
                        tweets.append(sent)
                        causal_labels.append(0)
#                        print("F: not both C + E in sentence, association: 0")
#                else:
#                    print("H: question in sentence")
            if row["Causal association"] == 0:
                print(sentences)
                print("H: should not have causality == 0")
#        else:
#            print("END: should not happen")

    
    return pd.DataFrame({"tweet" : tweets, "Causal association" : causal_labels})

trainingData = create_training_data(data)
print("N sentences:", trainingData.shape)
trainingData = trainingData[trainingData["tweet"].str.split(" ").str.len() > 3] # keep only sentence with more than 3 tokens
print("N sentences with > 3 words:", trainingData.shape)

N sentences: (9775, 2)
N sentences with > 3 words: (8232, 2)


In [8]:
# let's print a few example of our cleaned and normalized traingin dataset
trainingData.head()

,tweet,Causal association
0,"tonight , I learned my older girl will back he...",0.0
1,⬇ ️ ⬇ ️ ⬇ ️ THIS ⬇ ️ ⬇ ️ ⬇ ️ My wife has type ...,0.0
2,I'm a trans woman .,0.0
3,"Both of us could use a world where "" brave and...",0.0
4,"Make a world where people can just be , withou...",0.0


In [9]:
trainingData["Causal association"].value_counts()

0.0    7219
1.0    1013
Name: Causal association, dtype: int64

### Training

In [10]:
text = trainingData["tweet"].map(normalizeTweet).values.tolist()
labels = trainingData["Causal association"].values.tolist()
# first split the data into traingin and testing label in the ratio of 80:20
train_texts, test_texts, train_labels, test_labels = train_test_split(text, labels, test_size=0.2)
# split the new training data (80% of actual data) to get train and validation set
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2)



print("Train: {}".format(len(train_texts)))
print("Val: {}".format(len(val_texts)))
print("Test: {}".format(len(test_texts)))


Train: 5268
Val: 1317
Test: 1647


In [11]:
# train_texts[1]

In [12]:
# Transform labels + encodings into Pytorch DataSet object (including __len__, __getitem__)
class TweetDataSet(torch.utils.data.Dataset):
    def __init__(self, text, labels, tokenizer):
        self.text = text
        self.labels = labels
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.text, padding=True, truncation=True, return_token_type_ids=True)
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]
        return {
                "input_ids" : torch.tensor(ids[idx], dtype=torch.long)
              , "attention_mask" : torch.tensor(mask[idx], dtype=torch.long)
              , "token_type_ids" : torch.tensor(token_type_ids[idx], dtype=torch.long)
              , "labels" : torch.tensor(self.labels[idx], dtype=torch.long)
        }      

    def __len__(self):
        return len(self.labels)

    
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

train_dataset = TweetDataSet(train_texts, train_labels, tokenizer)
val_dataset = TweetDataSet(val_texts, val_labels, tokenizer)
test_dataset = TweetDataSet(test_texts, test_labels, tokenizer)
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

# put data to batches (iterables)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


5268
1317
1647


In [13]:
# !jupyter nbextension enable --py widgetsnbextension

In [14]:
# # from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
# # from transformers import AutoModelForSequenceClassification
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef

# def compute_metrics(pred, labels):
#     #labels = pred.label_ids
#     #preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
#     acc = accuracy_score(labels, pred)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }



# class CausalityBERT(torch.nn.Module):
#     """ Model Bert"""
#     def __init__(self):
#         super(CausalityBERT, self).__init__()
#         self.num_labels = 2
#         self.bert = transformers.BertModel.from_pretrained("vinai/bertweet-base")
#         self.dropout = torch.nn.Dropout(0.3)
#         self.linear = torch.nn.Linear(768, self.num_labels)
#         # softmax layer missing ? -> Vivek
        
#     def forward(self, input_ids, attention_mask, token_type_ids):
#         _, output_1 = self.bert(input_ids, attention_mask = attention_mask, token_type_ids=token_type_ids, return_dict=False)
#         output_2 = self.dropout(output_1)
#         logits = self.linear(output_2)
#         return logits


# ## Model parameters
# batchsize_train = 16
# lr = 5e-5
# adam_eps = 1e-8
# epochs = 3 
# num_warmup_steps = 0
# num_training_steps = len(train_loader)*epochs

# # Store our loss and accuracy for plotting
# train_loss_set = []
# learning_rate = []


# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# #model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base")
# model = CausalityBERT()
# model.to(device)

# # fine-tune only the task-specific parameters -> Vivek? 
# for param in model.bert.parameters():
#     param.requires_grad = False
    
# model.to(device)
# model.train() # set model to training mode


# optim = AdamW(model.parameters(), lr=lr, eps=adam_eps)
# scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps) # scheduler with a linearly decreasing learning rate from the initial lr set in the optimizer to 0; after a warmup period durnig which it increases linearly from 0 to the initial lr set in the optimizer
# loss_fn = CrossEntropyLoss()

# for epoch in trange(1, epochs+1, desc='Epoch'):
#     print("<" + "="*22 + F" Epoch {epoch} "+ "="*22 + ">")
    
#     #calculate total loss for this epoch
#     batch_loss = 0
    
#     for batch in tqdm(train_loader):
#         #print("batch:", batch)
#         optim.zero_grad() # gradients get accumulated by default -> clear previous accumulated gradients
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         token_type_ids = batch["token_type_ids"].to(device)
#         labels = batch['labels'].to(device)
#         #logits = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) # forward pass
#         logits = model(**{"input_ids":input_ids, "attention_mask":attention_mask, "token_type_ids":token_type_ids}) # forward pass
        
        
        
        
#         loss = loss_fn(logits, labels)
#         print("loss:", loss)
#         #loss = outputs[0]
#         loss.backward() # backward pass
#         optim.step()    # update parameters and take a step up using the computed gradient
#         scheduler.step()# update learning rate scheduler
#         batch_loss += loss.item() # update tracking variables
        
#     avg_train_loss = batch_loss / len(train_loader) # calculate avg loss over training data

#     # store the current learning rate
#     for param_group in optim.param_groups:
#         print("\n\tCurrent Learning rate: ", param_group['lr'])
#         learning_rate.append(param_group['lr'])
    
#     train_loss_set.append(avg_train_loss)
#     print(F'\n\tAverage Training loss: {avg_train_loss}')

    
#     ## ---- Validation ------
#     model.eval() # put model in evaluation mode for validation set
    
#     eval_accuracy, eval_mcc_accuracy, nb_eval_steps = 0, 0, 0 # Tracking variables
#     val_accuracy = []
#     val_loss = []
#     val_acc = []
#     val_prec = []
#     val_rec = []
#     val_f1 = []
    
    
#     # Evaluate data for one epoch
#     for batch in tqdm(validation_loader):
#         batch = tuple(batch[t].to(device) for t in batch)      # batch to GPU
#         b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch     # unpack inputs from dataloader
        
#         with torch.no_grad(): # tell model not to compute or store gradients -> saves memory + speeds up validation
#           #  logits = model(b_inputs_ids, attention_mask=b_input_mask, token_type_ids=b_token_type_ids) # forward pass, calculates logit predictions
#           logits = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions
            
#         loss = loss_fn(logits, b_labels)
#         val_loss.append(loss.item())
        
#         # move logits and labels to CPU
#         logits = logits.detach().to('cpu').numpy()
#         label_ids = b_labels.to('cpu').numpy()
        
#         pred_flat = np.argmax(logits, axis=1).flatten() # convert logits to list of predicted labels
#         labels_flat = label_ids.flatten()
#         eval_accuracy += accuracy_score(labels_flat, pred_flat)
#         eval_mcc_accrucay += matthews_corrcoef(labels_flat, pred_flat)        
#         metrics = compute_metrics(pred_flat, labels_flat)
#         val_acc.append(metrics["accuracy"])
#         val_prec.append(metrics["precision"])
#         val_rec.append(metrics["recall"])
#         val_f1.append(metrics["f1"])
#         nb_eval_steps += 1
        
#     print(F'\n\tValidation Accuracy: {eval_accuracy / nb_eval_steps}')
#     print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy / nb_eval_steps}')
#     print(F'\n\tValidation loss: {np.mean(val_loss)}')
#     print(F'\n\tValidation acc: {np.mean(val_acc)}')
#     print(F'\n\tValidation prec: {np.mean(val_prec)}')
#     print(F'\n\tValidation rec: {np.mean(val_rec)}')
#     print(F'\n\tValidation f1: {np.mean(val_f1)}')

    


In [15]:
# from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
# from transformers import AutoModelForSequenceClassification

## we are measuring weighted metrics - as our dataset is unbalanced 
# Calculate metrics for each label, and find their average weighted by support
#(the number of true instances for each label). 
#This alters ‘macro’ to account for label imbalance; 
# it can result in an F-score that is not between precision and recall.


from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef

def compute_metrics(pred, labels):
    #labels = pred.label_ids
    #preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='weighted')
    acc = accuracy_score(labels, pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



class CausalityBERT(torch.nn.Module):
    """ Model Bert"""
    def __init__(self):
        super(CausalityBERT, self).__init__()
        self.num_labels = 2
        self.bert = transformers.BertModel.from_pretrained("vinai/bertweet-base")
        self.dropout = torch.nn.Dropout(0.3)
        self.linear1 = torch.nn.Linear(768, 256)
        self.linear2 = torch.nn.Linear(256, self.num_labels)
        self.softmax = torch.nn.Softmax(-1)
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        _, output_1 = self.bert(input_ids, attention_mask = attention_mask, token_type_ids=token_type_ids, return_dict=False) # if output 1 is our cls token        
        output_2 = self.dropout(output_1)
        output_3 = self.linear1(output_2)  
        output_4 = self.dropout(output_3)
        output_5 = self.linear2(output_4)
        logit = self.softmax(output_5)
        return logit



    


In [16]:
# help(torch.nn.Softmax)

In [17]:
## Model parameters
batchsize_train = 16
lr = 5e-5
adam_eps = 1e-8
epochs = 35
num_warmup_steps = 0
num_training_steps = len(train_loader)*epochs

# Store our loss and accuracy for plotting : where is accuracy : it is loss an dlearning rate 
train_loss_set = []
learning_rate = []


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = CausalityBERT()
model.to(device)

# fine-tune only the task-specific parameters -> Vivek? 
for param in model.bert.parameters():
    param.requires_grad = False


optim = AdamW(model.parameters(), lr=lr, eps=adam_eps)
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps) # scheduler with a linearly decreasing learning rate from the initial lr set in the optimizer to 0; after a warmup period durnig which it increases linearly from 0 to the initial lr set in the optimizer
loss_fn = CrossEntropyLoss()



You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing BertModel: ['roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.11.attention.self.key.bias', 'roberta.encoder.layer.6.attention.output.LayerNorm.weight', 'roberta.encoder.layer.7.intermediate.dense.bias', 'roberta.encoder.layer.4.attention.self.key.bias', 'roberta.encoder.layer.7.attention.self.value.bias', 'roberta.encoder.layer.8.attention.self.query.bias', 'roberta.encoder.layer.8.attention.self.value.weight', 'roberta.encoder.layer.9.intermediate.dense.weight', 'lm_head.bias', 'roberta.encoder.layer.8.intermediate.dense.weight', 'roberta.encoder.layer.11.attention.self.value.weight', 'roberta.encoder.layer.9.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.7.at

In [32]:



for epoch in trange(1, epochs+1, desc='Epoch'):
    print("<" + "="*22 + F" Epoch {epoch} "+ "="*22 + ">")
    
    #calculate total loss for this epoch
#     epoch_loss = 0
    
    ########### training eval metrics #############################
    tr_accuracy, tr_mcc_accuracy, nb_tr_steps = 0, 0, 0 # Tracking variables
    train_accuracy = []
    train_loss = []
    train_acc = []
    train_prec = []
    train_rec = []
    train_f1 = []
    
    ###################################################
    
    for batch in tqdm(train_loader):
        #print("batch:", batch)
        optim.zero_grad() # gradients get accumulated by default -> clear previous accumulated gradients
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch['labels'].to(device)
        #logits = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) # forward pass
#         outputs = model(**{"input_ids":input_ids, "attention_mask":attention_mask, "token_type_ids":token_type_ids}) # forward pass
        ###########################################################################
        model.train()
        logits = model(**{"input_ids":input_ids, "attention_mask":attention_mask, "token_type_ids":token_type_ids}) # forward pass
        #############################################################################
        loss = loss_fn(logits, labels)
        print("loss:", loss)
        #loss = outputs[0]
        loss.backward() # backward pass
        optim.step()    # update parameters and take a step up using the computed gradient
        scheduler.step()# update learning rate scheduler
#         epoch_loss += loss.item() # update tracking variables
        train_loss.append(loss.item())
        
    
    ############# Training Accuracy Measure ##########################################
#         loss = loss_fn(logits, b_labels)
#         val_loss.append(loss.item())

        # move logits and labels to CPU
        logits = logits.detach().to('cpu').numpy()
        label_ids = labels.to('cpu').numpy()

        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()

        tr_accuracy += accuracy_score(labels_flat, pred_flat) # this is same as metric accuracy 
        tr_mcc_accuracy += matthews_corrcoef(labels_flat, pred_flat)  


        metrics = compute_metrics(pred_flat, labels_flat)
#         print(metrics)
        
        
#         train_loss.append(loss)
        train_acc.append(metrics["accuracy"])
        train_prec.append(metrics["precision"])
        train_rec.append(metrics["recall"])
        train_f1.append(metrics["f1"])
        
#         batch_loss
        nb_tr_steps += 1
        
#     batch_loss = np.mean(train_loss)
#     train_loss_set.append(batch_loss)
        
    
#     print(F'\n\tTraining Accuracy: {tr_accuracy / nb_tr_steps}') # accuracy is calculated twice 
    print(F'\n\tTraining MCC Accuracy: {tr_mcc_accuracy / nb_tr_steps}') # eval M
    
#     train_loss_set.append(batch_loss) # it should be either loss or batch_loss
    
    
#     train_loss = train_loss.to('cpu').numpy()
    print(F'\n\tTrain loss: {np.mean(train_loss)}')
    print(F'\n\ttrain acc: {np.mean(train_acc)}')
    print(F'\n\ttraining prec: {np.mean(train_prec)}')
    print(F'\n\ttraining rec: {np.mean(train_rec)}')
    print(F'\n\ttraining f1: {np.mean(train_f1)}')
    
    
    ###################################################################################
    #     avg_train_loss = batch_loss / len(train_loader) # calculate avg loss over training data : we don't need it as you did it for each sample
    #     train_loss_set.append(avg_train_loss)
    #     print(F'\n\tAverage Training loss: {avg_train_loss}')
    
#     train_loss_set.append(train_loss)
#     print(F'\n\tAverage Training loss per epoch: {train_loss}')


    # store the current learning rate
    for param_group in optim.param_groups:
        print("\n\tCurrent Learning rate: ", param_group['lr'])
        learning_rate.append(param_group['lr'])
    


    
    ## ---- Validation ------
#     model.eval() # put model in evaluation mode for validation set
    
    eval_accuracy, eval_mcc_accuracy, nb_eval_steps = 0, 0, 0 # Tracking variables
    val_accuracy = []
    val_loss = []
    val_acc = []
    val_prec = []
    val_rec = []
    val_f1 = []
    
    
    # Evaluate data for one epoch
    for batch in tqdm(validation_loader):
        batch = tuple(batch[t].to(device) for t in batch)      # batch to GPU
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch     # unpack inputs from dataloader
        
        with torch.no_grad(): # tell model not to compute or store gradients -> saves memory + speeds up validation
          #  logits = model(b_inputs_ids, attention_mask=b_input_mask, token_type_ids=b_token_type_ids) # forward pass, calculates logit predictions
#             outputs = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions 
            ##################################################################################
            model.eval()
            logits = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions 

                        
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())
        
        # move logits and labels to CPU
        logits = logits.detach().to('cpu').numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        
        eval_accuracy += accuracy_score(labels_flat, pred_flat)
        eval_mcc_accuracy += matthews_corrcoef(labels_flat, pred_flat)  
        
        
        metrics = compute_metrics(pred_flat, labels_flat)
        val_acc.append(metrics["accuracy"])
        val_prec.append(metrics["precision"])
        val_rec.append(metrics["recall"])
        val_f1.append(metrics["f1"])
        nb_eval_steps += 1
        
#     print(F'\n\tValidation Accuracy: {eval_accuracy / nb_eval_steps}')
    print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy / nb_eval_steps}') # eval M
    
#     val_loss = val_loss.to('cpu').numpy()
    print(F'\n\tValidation loss: {np.mean(val_loss)}')
    print(F'\n\tValidation acc: {np.mean(val_acc)}')
    print(F'\n\tValidation prec: {np.mean(val_prec)}')
    print(F'\n\tValidation rec: {np.mean(val_rec)}')
    print(F'\n\tValidation f1: {np.mean(val_f1)}')


  0%|          | 0/83 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>


<ipython-input-12-50b3d035d7dd>:17: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  , "labels" : torch.tensor(self.labels[idx], dtype=torch.long)


loss: 


  1%|          | 1/83 [00:22<30:57, 22.66s/it]

...

tensor(0.4070, device='cuda:0', grad_fn=<NllLossBackward>)



100%|██████████| 83/83 [30:08<00:00, 21.79s/it]

  0%|          | 0/42 [00:00<?, ?it/s]

loss: tensor(0.4133, device='cuda:0', grad_fn=<NllLossBackward>)

	Training MCC Accuracy: 0.0

	Train loss: 0.4353540528969592

	train acc: 0.8779367469879519

	training prec: 0.7722955101656627

	training rec: 0.8779367469879519

	training f1: 0.8213389265373052

	Current Learning rate:  0.0



Epoch: 100%|██████████| 35/35 [18:07:52<00:00, 1864.92s/it]


	Validation MCC Accuracy: 0.0

	Validation loss: 0.4256184094008945

	Validation acc: 0.8876488095238095

	Validation prec: 0.7907598586309523

	Validation rec: 0.8876488095238095

	Validation f1: 0.8356668417231058


In [33]:
# train_loss.numpy()
# train_loss


In [34]:
# # Saving models
# torch.save(model.state_dict(), "finetuned.pth")

# #load models
# model = Neural

In [50]:
## saving the model 
torch.save(model.state_dict(), "finetuned-35-epochs.pth")

In [51]:
# loading the locally saved model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = CausalityBERT()
model.load_state_dict(torch.load("finetuned-35-epochs.pth"))
## Move the model to the GPU 
model.to(device)
model.eval() # gettign in the eval mode 



You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing BertModel: ['roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.11.attention.self.key.bias', 'roberta.encoder.layer.6.attention.output.LayerNorm.weight', 'roberta.encoder.layer.7.intermediate.dense.bias', 'roberta.encoder.layer.4.attention.self.key.bias', 'roberta.encoder.layer.7.attention.self.value.bias', 'roberta.encoder.layer.8.attention.self.query.bias', 'roberta.encoder.layer.8.attention.self.value.weight', 'roberta.encoder.layer.9.intermediate.dense.weight', 'lm_head.bias', 'roberta.encoder.layer.8.intermediate.dense.weight', 'roberta.encoder.layer.11.attention.self.value.weight', 'roberta.encoder.layer.9.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.7.at

CausalityBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True

## evaluation on the test dataset 



In [38]:
## evaluation on the test dataset 

# test_accuracy, test_mcc_accuracy, nb_test_steps = 0, 0, 0 
test_mcc_accuracy, nb_test_steps = 0, 0,

# test_accuracy = []
test_loss = []
test_acc = []
test_prec = []
test_rec = []
test_f1 = []

for batch in tqdm(test_loader):
    batch = tuple(batch[t].to(device) for t in batch)      # batch to GPU
    b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch     # unpack inputs from dataloader
    with torch.no_grad():
        model.eval()
        logits = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions 
    
    
    loss = loss_fn(logits, b_labels)
    test_loss.append(loss.item())

    # move logits and labels to CPU
    logits = logits.detach().to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()

#     eval_accuracy += accuracy_score(labels_flat, pred_flat)
    test_mcc_accuracy += matthews_corrcoef(labels_flat, pred_flat)  


    metrics = compute_metrics(pred_flat, labels_flat)
    test_acc.append(metrics["accuracy"])
    test_prec.append(metrics["precision"])
    test_rec.append(metrics["recall"])
    test_f1.append(metrics["f1"])
    nb_test_steps += 1
    

# print(F'\n\tValidation Accuracy: {eval_accuracy / nb_eval_steps}')
print(F'\n\ttest MCC Accuracy: {test_mcc_accuracy / nb_test_steps}') # eval M
print(F'\n\ttest loss: {np.mean(test_loss)}')
print(F'\n\ttest acc: {np.mean(test_acc)}')
print(F'\n\ttest prec: {np.mean(test_prec)}')
print(F'\n\ttest rec: {np.mean(test_rec)}')
print(F'\n\ttest f1: {np.mean(test_f1)}')

  0%|          | 0/52 [00:00<?, ?it/s]<ipython-input-12-50b3d035d7dd>:17: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  , "labels" : torch.tensor(self.labels[idx], dtype=torch.long)
100%|██████████| 52/52 [02:47<00:00,  3.22s/it]


	test MCC Accuracy: 0.0

	test loss: 0.44631937260811144

	test acc: 0.8669471153846153

	test prec: 0.7549646935096154

	test rec: 0.8669471153846153

	test f1: 0.8062038738692032


### printig some variabels to look into 

In [40]:
train_loss


[0.46953466534614563,
 0.4695369601249695,
 0.42266905307769775,
 0.3914310932159424,
 0.40704697370529175,
 0.42267104983329773,
 0.4070471227169037,
 0.360171914100647,
 0.4539085030555725,
 0.4382795989513397,
 0.4070567488670349,
 0.46954286098480225,
 0.4226679801940918,
 0.4226563274860382,
 0.45392122864723206,
 0.43829724192619324,
 0.4070553779602051,
 0.48515892028808594,
 0.45390549302101135,
 0.3757953643798828,
 0.4070453643798828,
 0.40703269839286804,
 0.4226720929145813,
 0.5007822513580322,
 0.407050758600235,
 0.45391330122947693,
 0.4851546883583069,
 0.40706220269203186,
 0.3758067190647125,
 0.42265719175338745,
 0.42266646027565,
 0.39142996072769165,
 0.4695398509502411,
 0.42267733812332153,
 0.5007687211036682,
 0.3758033514022827,
 0.43828701972961426,
 0.4226643741130829,
 0.3601856529712677,
 0.4382989704608917,
 0.48515984416007996,
 0.40704599022865295,
 0.4382978081703186,
 0.4539179801940918,
 0.46953898668289185,
 0.4070377051830292,
 0.453912615776062,

In [41]:
train_acc 

[0.84375,
 0.84375,
 0.890625,
 0.921875,
 0.90625,
 0.890625,
 0.90625,
 0.953125,
 0.859375,
 0.875,
 0.90625,
 0.84375,
 0.890625,
 0.890625,
 0.859375,
 0.875,
 0.90625,
 0.828125,
 0.859375,
 0.9375,
 0.90625,
 0.90625,
 0.890625,
 0.8125,
 0.90625,
 0.859375,
 0.828125,
 0.90625,
 0.9375,
 0.890625,
 0.890625,
 0.921875,
 0.84375,
 0.890625,
 0.8125,
 0.9375,
 0.875,
 0.890625,
 0.953125,
 0.875,
 0.828125,
 0.90625,
 0.875,
 0.859375,
 0.84375,
 0.90625,
 0.859375,
 0.921875,
 0.75,
 0.921875,
 0.859375,
 0.828125,
 0.921875,
 0.875,
 0.90625,
 0.84375,
 0.84375,
 0.90625,
 0.84375,
 0.796875,
 0.84375,
 0.765625,
 0.890625,
 0.921875,
 0.90625,
 0.859375,
 0.875,
 0.875,
 0.921875,
 0.890625,
 0.828125,
 0.921875,
 0.875,
 0.90625,
 0.84375,
 0.859375,
 0.90625,
 0.875,
 0.875,
 0.875,
 0.828125,
 0.90625,
 0.9]

In [42]:
train_prec

[0.7119140625,
 0.7119140625,
 0.793212890625,
 0.849853515625,
 0.8212890625,
 0.793212890625,
 0.8212890625,
 0.908447265625,
 0.738525390625,
 0.765625,
 0.8212890625,
 0.7119140625,
 0.793212890625,
 0.793212890625,
 0.738525390625,
 0.765625,
 0.8212890625,
 0.685791015625,
 0.738525390625,
 0.87890625,
 0.8212890625,
 0.8212890625,
 0.793212890625,
 0.66015625,
 0.8212890625,
 0.738525390625,
 0.685791015625,
 0.8212890625,
 0.87890625,
 0.793212890625,
 0.793212890625,
 0.849853515625,
 0.7119140625,
 0.793212890625,
 0.66015625,
 0.87890625,
 0.765625,
 0.793212890625,
 0.908447265625,
 0.765625,
 0.685791015625,
 0.8212890625,
 0.765625,
 0.738525390625,
 0.7119140625,
 0.8212890625,
 0.738525390625,
 0.849853515625,
 0.5625,
 0.849853515625,
 0.738525390625,
 0.685791015625,
 0.849853515625,
 0.765625,
 0.8212890625,
 0.7119140625,
 0.7119140625,
 0.8212890625,
 0.7119140625,
 0.635009765625,
 0.7119140625,
 0.586181640625,
 0.793212890625,
 0.849853515625,
 0.8212890625,
 0.

In [43]:

train_rec

[0.84375,
 0.84375,
 0.890625,
 0.921875,
 0.90625,
 0.890625,
 0.90625,
 0.953125,
 0.859375,
 0.875,
 0.90625,
 0.84375,
 0.890625,
 0.890625,
 0.859375,
 0.875,
 0.90625,
 0.828125,
 0.859375,
 0.9375,
 0.90625,
 0.90625,
 0.890625,
 0.8125,
 0.90625,
 0.859375,
 0.828125,
 0.90625,
 0.9375,
 0.890625,
 0.890625,
 0.921875,
 0.84375,
 0.890625,
 0.8125,
 0.9375,
 0.875,
 0.890625,
 0.953125,
 0.875,
 0.828125,
 0.90625,
 0.875,
 0.859375,
 0.84375,
 0.90625,
 0.859375,
 0.921875,
 0.75,
 0.921875,
 0.859375,
 0.828125,
 0.921875,
 0.875,
 0.90625,
 0.84375,
 0.84375,
 0.90625,
 0.84375,
 0.796875,
 0.84375,
 0.765625,
 0.890625,
 0.921875,
 0.90625,
 0.859375,
 0.875,
 0.875,
 0.921875,
 0.890625,
 0.828125,
 0.921875,
 0.875,
 0.90625,
 0.84375,
 0.859375,
 0.90625,
 0.875,
 0.875,
 0.875,
 0.828125,
 0.90625,
 0.9]

In [44]:

train_f1

[0.7722457627118644,
 0.7722457627118644,
 0.8391012396694215,
 0.884400406504065,
 0.8616803278688524,
 0.8391012396694215,
 0.8616803278688524,
 0.93025,
 0.7943802521008404,
 0.8166666666666667,
 0.8616803278688524,
 0.7722457627118644,
 0.8391012396694215,
 0.8391012396694215,
 0.7943802521008404,
 0.8166666666666667,
 0.8616803278688524,
 0.750267094017094,
 0.7943802521008404,
 0.907258064516129,
 0.8616803278688524,
 0.8616803278688524,
 0.8391012396694215,
 0.728448275862069,
 0.8616803278688524,
 0.7943802521008404,
 0.750267094017094,
 0.8616803278688524,
 0.907258064516129,
 0.8391012396694215,
 0.8391012396694215,
 0.884400406504065,
 0.7722457627118644,
 0.8391012396694215,
 0.728448275862069,
 0.907258064516129,
 0.8166666666666667,
 0.8391012396694215,
 0.93025,
 0.8166666666666667,
 0.750267094017094,
 0.8616803278688524,
 0.8166666666666667,
 0.7943802521008404,
 0.7722457627118644,
 0.8616803278688524,
 0.7943802521008404,
 0.884400406504065,
 0.6428571428571428,
 0.8

In [45]:
val_accuracy

[]

In [46]:
val_loss

[0.46951550245285034,
 0.4382668137550354,
 0.4382668733596802,
 0.4070178270339966,
 0.46951544284820557,
 0.43826672434806824,
 0.4382666051387787,
 0.4695158004760742,
 0.5320134162902832,
 0.4070179760456085,
 0.4070177674293518,
 0.438266783952713,
 0.469515323638916,
 0.4070178270339966,
 0.3757689297199249,
 0.5632620453834534,
 0.3132708668708801,
 0.4070178270339966,
 0.438267320394516,
 0.40701788663864136,
 0.3757689595222473,
 0.46951550245285034,
 0.4070180654525757,
 0.4382669925689697,
 0.4382668435573578,
 0.40701743960380554,
 0.37576913833618164,
 0.34452033042907715,
 0.34452003240585327,
 0.4695158302783966,
 0.3757689297199249,
 0.3757689595222473,
 0.4070180058479309,
 0.5007646083831787,
 0.43826672434806824,
 0.37576913833618164,
 0.5007641911506653,
 0.5007648468017578,
 0.40701791644096375,
 0.4695156514644623,
 0.4070180058479309,
 0.31327152252197266]

In [47]:
val_acc

[0.84375,
 0.875,
 0.875,
 0.90625,
 0.84375,
 0.875,
 0.875,
 0.84375,
 0.78125,
 0.90625,
 0.90625,
 0.875,
 0.84375,
 0.90625,
 0.9375,
 0.75,
 1.0,
 0.90625,
 0.875,
 0.90625,
 0.9375,
 0.84375,
 0.90625,
 0.875,
 0.875,
 0.90625,
 0.9375,
 0.96875,
 0.96875,
 0.84375,
 0.9375,
 0.9375,
 0.90625,
 0.8125,
 0.875,
 0.9375,
 0.8125,
 0.8125,
 0.90625,
 0.84375,
 0.90625,
 1.0]

In [48]:
val_prec

[0.7119140625,
 0.765625,
 0.765625,
 0.8212890625,
 0.7119140625,
 0.765625,
 0.765625,
 0.7119140625,
 0.6103515625,
 0.8212890625,
 0.8212890625,
 0.765625,
 0.7119140625,
 0.8212890625,
 0.87890625,
 0.5625,
 1.0,
 0.8212890625,
 0.765625,
 0.8212890625,
 0.87890625,
 0.7119140625,
 0.8212890625,
 0.765625,
 0.765625,
 0.8212890625,
 0.87890625,
 0.9384765625,
 0.9384765625,
 0.7119140625,
 0.87890625,
 0.87890625,
 0.8212890625,
 0.66015625,
 0.765625,
 0.87890625,
 0.66015625,
 0.66015625,
 0.8212890625,
 0.7119140625,
 0.8212890625,
 1.0]

In [ ]:
val_rec

In [ ]:
val_f1

In [ ]:
test_accuracy

In [ ]:
test_loss

In [ ]:
test_acc

In [ ]:
test_prec

In [ ]:
test_rec

In [ ]:
test_f1